In [1]:
import numpy as np
import spatialmath as sm
import quaternion
import rospy
# set printing and plotting options
np.set_printoptions(precision=5, suppress=True)
rospy.init_node('Moving')
import hrr_cobot_robot as hrr_rob
cobot = hrr_rob.HrrCobotControl.from_ros("/hrr_cobot")

         at line 278 in /tmp/binarydeb/ros-noetic-tf2-0.7.5/src/buffer_core.cpp


[rosout] [/Moving] Initialize MATLAB-Motion planner @ /home/hrr_cobot/_ros/hr_recycler_ws/src/planner/src
[rosout] [/Moving] connected to Matlab
Shutting down global node /matlab_global_node_92072 with NodeURI http://192.168.2.87:34493/
The value of the ROS_IP environment variable, 192.168.2.87, will be used to set the advertised address for the ROS node.
Initializing global node /matlab_global_node_82035 with NodeURI http://192.168.2.87:40201/
[rosout] [/Moving] MATLAB-Initialization done


pybullet build time: May  8 2021 05:48:13


[rosout] some functions are not yet compiled. Expect delays upon first call


In [ ]:
np.set_printoptions(precision=5, suppress=True)

In [ ]:
#cobot.change_tool("shaftgrinder")
#cobot.change_tool("screwdriver")
cobot.change_tool("vacuum")
#All other tools x and y are same as end of robot arm
#cobot.change_tool("nothing")

In [ ]:
cobot.change_tool("vacuum")

In [ ]:
cobot.tool

In [ ]:
#Position of end effector (end of robot arm)
cobot.T_B_E_robot, cobot.T_B_E_robot.t

In [ ]:
#Position of tip
cobot.T_B_C_robot, cobot.T_B_C_robot.t

In [ ]:
cobot.open_tool_changer()

In [ ]:
cobot.close_tool_changer()

### Move in cartesian base frame x y z

In [ ]:
cobot.T_B_E_robot

In [ ]:
cobot.T_B_E_robot

In [2]:
cartesian_difference = np.r_[-0, -0.1, 0.0]

In [3]:
cobot.goTo(sm.SE3(cartesian_difference)@cobot.T_B_E_robot,v_max=0.03)

In [ ]:
cobot.goTo(sm.SE3(cartesian_difference)@cobot.T_B_E_robot,v_max=0.12)
cobot.goTo(sm.SE3(-cartesian_difference)@cobot.T_B_E_robot,v_max=0.12)

### Move forward or backwards with end effector

In [ ]:
distance = 0.05 #Positive means we move forward, negative means we move backward, both in EE frame

In [ ]:
cobot.goTo(cobot.T_B_E_robot@sm.SE3([0,0,distance]))

### Move to calibration pose orientation but with desired joint 1 angle

In [ ]:
joint_1_angle_deg =0 #in degrees

In [ ]:
qdes = np.r_[np.deg2rad(joint_1_angle_deg), 0., -np.pi / 2.0, 0., np.pi / 2.0, np.pi / 2.0]
cobot.move_to_joint_pose(qdes, stochastic = False)

### Move to calibration pose 

In [4]:
q_neutral = np.r_[1.34109, 0.09328, -1.44435, -0.01627, 0.67686, -0.00009]
cobot.move_to_joint_pose(q_neutral, stochastic=False)

In [ ]:
cobot.move_to_joint_pose(cobot.q_calib, stochastic=False)

In [ ]:
cobot.change_tool("nothing")

In [ ]:
cobot.move_to_joint_pose(np.r_[0, 0, -1.5708, 0., 1.5708, 1.5708], stochastic=False)

In [ ]:
cobot.FT.reset_bias() 

In [ ]:
cobot.B_F_msr #if these values go positive x

In [ ]:
cobot.B_F_msr #if these values go negative x

In [ ]:
cobot.B_F_msr #perfect

In [ ]:
cobot.change_tool("wsg_50_dsa")

### Tilting

In [ ]:
v_test = np.zeros(6)
for t in range(300):
    v_test[5] = 2e-2
    cobot.update(u_cmd=v_test, u_cmd_frame="hrr_cobot.screwdriver_tip", sleep=True)
cobot.stop()

In [ ]:
cobot.init_sns_vel()
t0 = rospy.get_time()
while rospy.get_time()-t0<1:
    cobot.update(u_cmd=np.r_[0,0,0,0.3,0,0], u_cmd_frame="hrr_cobot.ee_link",sleep=True)
cobot.stop()

In [ ]:
cobot.goTo(sm.SE3([0, 0, 0.1]) @ cobot.T_B_E_robot, v_max=0.05)
cobot.init_sns_vel()
t0 = rospy.get_time()
while rospy.get_time()-t0<1:
    cobot.update(u_cmd=np.r_[0,0,0,0.3,0,0], u_cmd_frame="hrr_cobot.ee_link",sleep=True)
cobot.stop()
cobot.goTo(sm.SE3([0, 0.107, 0.0]) @ cobot.T_B_E_robot, v_max=0.05)

In [ ]:
cobot.change_tool("vacuum")

In [ ]:
cobot.tool_controller.vacuum = False

In [ ]:
rospy.get_time()-t0

# STOP!!!!

In [ ]:
cobot.emergency_stop()
print(cobot.state)
cobot.reset()
print(cobot.state)

In [ ]:
from tqdm.notebook import tqdm, trange

In [ ]:
cobot.change_tool("screwdriver")

In [ ]:
for t in range(150):
        cobot.update(u_cmd=np.r_[0,0,0,-2e-1,0,0], u_cmd_frame="hrr_cobot.ee_link",sleep=True)

In [ ]:
cobot.T_B_E_robot

In [ ]:
cobot.T_B_E_robot

In [ ]:
cobot.init_sns_vel()
for t in range(150):
        cobot.update(u_cmd=np.r_[0,0,0,-2e-1,0,0], u_cmd_frame="hrr_cobot.ee_link",sleep=True)
cobot.stop()

# Testing FT with Compliant Mode

In [ ]:
cobot.set_compliant(np.r_[1, 1, np.zeros(3), 1], use_py_vel=True)

In [ ]:
cobot.set_py_hybrid_force_vel_command(B_F_des=np.r_[0.0, 0.0, -5, np.zeros(3)], 
                                          K_f=8e-4, K_t=5e-3, scale_pos=0.0, scale_rot=0.0, 
                                      wrench_dir=np.r_[1.0, 1.0, 1.0, np.ones(3)],
                                      vel_dir=np.zeros(6))

In [ ]:
print(cobot)

In [ ]:
 # commented for safety reasons -> outcomment for testing
#rospy.sleep(5.0)
T = int(10* cobot.hz)
for t in range(T):
    cobot.update()

In [ ]:
cobot.B_F_ext

In [ ]:
#Check if force values are not crazily high (above 40 or 50)
cobot.B_F_msr #gives back force in x y z and torque around x y z axis

In [ ]:
cobot.FT.reset_bias()

# Testing

In [ ]:
import numpy as np
import spatialmath as sm

In [ ]:
fixed_pose = sm.SE3()
fixed_pose_straight = sm.SE3()
fixed_pose_straight.A[:4, :4] = [[0.81119, 0.00089, 0.58478, 0.58263],
                                 [-0.58245, -0.08804, 0.80809, -0.23876],
                                 [0.0522, -0.99612, -0.0709, 0.50576],
                                 [0., 0., 0., 1.]]

In [ ]:
fixed_pose_straight.R, (sm.SE3(0.1 * np.r_[1/8, 1/3, 1/2]) @ fixed_pose_straight).R

In [ ]:
cobot.stochastic_move_to_pose(fixed_pose_straight)

# Open and close tool changer

In [ ]:
cobot.open_tool_changer()

In [ ]:
cobot.close_tool_changer()

## Neutral pose and old Calibration pose

In [ ]:
cobot.tool

In [ ]:
cobot.change_tool("wsg_50_dsa")

In [ ]:
q_neutral = np.r_[1.34109, 0.09328, -1.44435, -0.01627, 0.67686, -0.00009]

In [ ]:
cobot.move_to_joint_pose(q_neutral, stochastic=False)

In [ ]:
cobot.goTo(cobot.FK(q_neutral), v_max=0.02)

In [ ]:
cobot.move_to_joint_pose(cobot.q_calib, stochastic=False)

In [ ]:
q_des = np.array([-np.deg2rad(0), 0, -1.5708, 0., 1.5708, 1.5708])


### Elamp cutting orientation

In [ ]:
cobot.move_to_joint_pose(
    np.r_[0.2612346411, 0.250723213, -2.4299004078, -0.2601425946, -1.1549400091, 1.7408881187], stochastic= False)

# Poses for unscrewing

In [ ]:
cobot.change_tool("shaftgrinder")

In [ ]:
cobot.tool

In [ ]:
cobot.T_B_C_robot.t

In [ ]:
#Proposal for flat panel display if y>0
beginning_pose1 = np.r_[-0.79806,  0.61373, -1.84638, -1.53851, -1.56184,  2.24939]

In [ ]:
#Proposal for flat panel display if y<0
beginning_pose2 = np.r_[ 0.8735 ,  0.55755, -1.96375,  1.63136, -1.63044,  0.94922]

In [ ]:
cobot.move_to_joint_pose(beginning_pose2, stochastic=False)

In [ ]:
#New pose for microwave that is higher
beginning_pose3 = np.r_[ 0.86289,  0.54018, -1.66603,  1.60357, -1.63773,  0.63349]

In [ ]:
#Microwave tilted
beginning_pose = np.r_[ 0.72449,  0.59531, -1.42626,  1.4668 , -1.53804,  0.40122]

# Test normal vector stuff
This is how your orient the screwdriver use the vision normal, while keeping rest the same as much as possible. Test a bit more, then it can be implemented into unscrewing skill

In [ ]:
cobot.change_tool("screwdriver")

In [ ]:
start = sm.SE3()
start.A[:4,:4] = [[0.7042,    0.007712,  0.71,      0.494],     
  [-0.71,      0.01122,   0.7041,   -0.03627],   
  [-0.002539, -0.9999,    0.01338,   0.4907],    
   [0         ,0         ,0         ,1]]  
cobot.goTo(start)

In [ ]:
normal = np.r_[-0.13, -0.13, 0.98]
normal = normal/np.linalg.norm(normal)
T_des = sm.SE3(cobot.T_B_E_robot.t)
T_des.A[:3, :3] = sm.base.oa2r(np.r_[0,0,1], -normal)
T_des = T_des @ cobot.T_E_C_robot.inv()
T_des

In [ ]:
#This where tip would be
(T_des @ cobot.T_E_C_robot).t


In [ ]:
#We want tip at
cobot.T_B_C_robot.t

In [ ]:
#So the difference is
difference = cobot.T_B_C_robot.t - (T_des @ cobot.T_E_C_robot).t
difference

In [ ]:
#So we need to translate the correctly oriented pose by the difference
T_des_correct = sm.SE3(difference) @ T_des
T_des_correct, T_des_correct @ cobot.T_E_C_robot

In [ ]:
cobot.goTo(T_des_correct)

# Test force sensor shit

In [ ]:
def _quat_rotate_vec(q1, v):
    return (q1 * np.quaternion(0, *v) * q1.conjugate()).vec

In [ ]:
cobot.FT_F, cobot.B_F_msr

In [ ]:
bfmsr = np.r_[_quat_rotate_vec(cobot.sns_quat, cobot.R_FT_E.T @ cobot.FT_F[0:3]),
             _quat_rotate_vec(cobot.sns_quat, cobot.R_FT_E.T @ cobot.FT_F[3:6])]
bfmsr

In [ ]:
cobot.sns_quat

In [ ]:
cobot.T_B_E_robot.R @ cobot.FT_F[0:3]

# whatever testing

In [ ]:
cobot.tool

In [ ]:
cobot.T_B_C_robot

In [ ]:
cobot.T_B_C_robot

In [ ]:
cobot.change_tool("shaftgrinder")

In [ ]:
cobot.move_to_joint_pose(np.r_[ 0.55734, 0.11429, -1.78829, 0.6457, -0.42092, 0.969], stochastic=False)

# Vacuum grasping

In [ ]:
cobot.T_B_E_robot.t

In [ ]:
object_center_t = np.r_[0.65,  0.,  0.1]

In [ ]:
cobot.move_to_joint_pose(cobot.q_calib, stochastic = False)

In [ ]:
from comau_msgs.srv import SetIO, SetIORequest
srv = rospy.ServiceProxy("/hrr_cobot/set_digital_io", SetIO)
srv(SetIORequest(pin=300, state=True))                    

In [ ]:
srv(SetIORequest(pin=300, state=False))

In [ ]:
cobot.open_tool_changer()

In [ ]:
cobot.close_tool_changer()

In [ ]:
cobot.change_tool("shaftgrinder")

In [ ]:
cobot.q

In [ ]:
cobot.T_B_E_robot.t - ee2tip

In [ ]:
grasp_mode = 0

ee2tip = np.r_[0, 0, 0.20018]
if grasp_mode == -1:
    ee2tip += np.r_[0, -0.053, 0]
elif grasp_mode == 1:
    ee2tip += np.r_[0, 0.053, 0]

In [ ]:
q_des = np.array([-np.deg2rad(0), 0, -1.5708, 0., 1.5708, 1.5708])
R_des = cobot.FK(q_des).R

grasp_pose = sm.SE3(object_center_t + ee2tip)
grasp_pose.A[0:3, 0:3] = R_des
grasp_pose

In [ ]:
q_des = np.array([-np.deg2rad(90), 0, -1.5708, 0., 1.5708, 1.5708])

In [ ]:
safety_dist = 0.05
below_dist = -0.05
pre_pose = grasp_pose @ sm.SE3([0, 0, -safety_dist])
below_pose = grasp_pose @ sm.SE3([0, 0, -below_dist])

In [ ]:
cobot.tool_controller.vacuum = False

In [ ]:
cobot.move_to_pose(below_pose, v_max=0.01)
F0 = np.copy(cobot.B_F_msr[0:3])
for t in range(int(100 * cobot.hz)):
    if cobot.state is None:
        print("done")
        break
    elif cobot.state == "error":
        print("error")
        break
    if ((cobot.B_F_msr[:3] - F0) - cobot.FT.noise[:3]).T @ np.r_[0,0,1] >= 10:
        print("contact")
        cobot.tool_controller.vacuum = True
        # Enable vacuum
        rospy.sleep(0.5)
        break
    cobot.update()


In [ ]:
cobot.goTo(pre_pose, v_max=0.05) 

In [ ]:
cobot.change_tool("vacuum")

In [ ]:
cobot.tool_controller.vacuum = False

In [ ]:
cobot.tool_controller.vacuum = True

In [ ]:
cobot.change_tool("wsg_50")